# lhorizon example 1: where was GALEX in May 2003?

Imagine that you are examining a portion of the observational data record of the GALEX space telescope from May 2003 and you realize that there is an anomaly that might be explicable by a barycentric time offset. A SPK SPICE kernel for GALEX may exist somewhere, but you do not know where. Horizons contains detailed information about the positions of many orbital and ground intrument platforms, and ```lhorizon``` can help you quickly figure out where GALEX was during this period.

This is a relatively short time period at relatively coarse resolution. If you realize that you need higher resolution or if you'd like to do larger queries -- ones with more than about 70K rows -- take a look at the bulk query functions in mars_sun_angle.ipynb.

In [ ]:
# run these imports if you'd like the code to function

import astropy.time as at

from lhorizon import LHorizon

In [ ]:
# horizons code for the SSB
solar_system_barycenter = '500@0'
coordinate_origin = solar_system_barycenter
# horizons knows the name "GALEX". Its Horizons numeric id, -127783, could also be used.
galex_horizons_id = "GALEX"
target_body_id = galex_horizons_id

# Time units are not consistent across different types of Horizons queries. in particular, 
# times for vectors queries are in TDB, which in this case is about 64 seconds later than UTC. 
# Conversion from UTC to TDB can be accomplished easily using astropy.time.
start = '2003-05-01T00:00:00'
stop = '2003-05-15T01:00:00'
step = "5m"

start_tdb = at.Time(start).tdb.iso
stop_tdb = at.Time(stop).tdb.iso

In [ ]:
# make a LHorizon with these values. 
galex_icrf = LHorizon(
    galex_horizons_id,
    coordinate_origin,
    epochs = {
        'start': start_tdb,
        'stop': stop_tdb,
        'step': step
    },
    query_type='VECTORS'
)
# fetch these data and concatenate them into a pandas dataframe.
# the LHorizon.table() method grabs a selection of columns from
# the Horizons response, regularizes units to meters and 
# seconds, and makes some column names clearer or more tractable. 
# if you want the full, unvarnished collection of values returned by Horizons
# with no modifications other than whitespace removal, 
# use the LHorizons.dataframe() method instead.
vector_table = galex_icrf.table()

# note that the coordinate system in this particular query is ICRF 
# of the most conventional kind -- measured from the solar system barycenter,
# geometric states uncorrected for light time or stellar aberration.

# columns are:
# time_tdb: time -- still in the TDB scale
# x, y, z: components of position vector in m
# vx, vy, vz: components of velocity vector in m/s
# dist: distance in m
# velocity: velocity in m/s

In [ ]:
vector_table

In [ ]:
# since this is a pandas dataframe, it can be easily manipulated in Python. If you'd rather work with it
# in some other way, it can also be easily written to CSV. 
vector_table.to_csv("vector_table " + start + " to " + stop + ".csv", index=None)